In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("USstatesCov19distancingpolicy.csv", encoding='latin1')

cols = ['StatePostal', 'StateName', 'StatePolicy','StateWide', 'VaccineExempt', 'PolicyCodingNotes', 
        'PolicySource', 'LastUpdated', 'LastUpdatedNotes', 'ReReviewed']

bar_df = df[(df.StatePolicy == 'BarRestrict') & (df.StateWide == 1)]
bar_df = bar_df[bar_df.DateIssued.values < 20210000]

bar_df[(bar_df.DateEnded.values > 20210000) & ~(np.isnan(bar_df.DateEnded.values))] = 20201231
bar_df[(bar_df.DateEased.values > 20210000) & ~(np.isnan(bar_df.DateEased.values))] = 20201231
bar_df[(bar_df.DateExpiry.values > 20210000) & ~(np.isnan(bar_df.DateExpiry.values))] = 20201231

Enacted_month = []
Enacted_day = []
Ended_month = []
Ended_day = []

for i in range(bar_df.shape[0]):
    row = bar_df.iloc[i]

    if np.isnan(row.DateEnacted):
        Enacted_month.append(int(str(row.DateIssued)[4:6])) 
        Enacted_day.append(int(str(row.DateIssued)[6:8])) 

    else:
        Enacted_month.append(int(str(row.DateEnacted)[4:6])) 
        Enacted_day.append(int(str(row.DateEnacted)[6:8])) 

    if np.isnan(row.DateEnded):
        if np.isnan(row.DateEased):

            if np.isnan(row.DateExpiry):
                Ended_month.append(row.DateExpiry) 
                Ended_day.append(row.DateExpiry) 
            else:
                Ended_month.append(int(str(row.DateExpiry)[4:6])) 
                Ended_day.append(int(str(row.DateExpiry)[6:8])) 

        else:
            Ended_month.append(int(str(row.DateEased)[4:6])) 
            Ended_day.append(int(str(row.DateEased)[6:8])) 

    else:
        Ended_month.append(int(str(row.DateEnded)[4:6])) 
        Ended_day.append(int(str(row.DateEnded)[6:8])) 

bar_df = bar_df[cols]

bar_df['EnactedMonth'] = Enacted_month
bar_df['EnactedDay'] = Enacted_day
bar_df['EndedMonth'] = Ended_month
bar_df['EndedDay'] = Ended_day

In [3]:
Tx_df = bar_df[bar_df.StatePostal == 'TX']
Tx_df

,StatePostal,StateName,StatePolicy,StateWide,VaccineExempt,PolicyCodingNotes,PolicySource,LastUpdated,LastUpdatedNotes,ReReviewed,EnactedMonth,EnactedDay,EndedMonth,EndedDay
1714,TX,Texas,BarRestrict,1,NaN,"Limited operations required (drive-thru, picku...",https://lrl.texas.gov/scanned/govdocs/Greg%20A...,20210118,Updated full entry,1.0,3,21,5.0,22.0
1731,TX,Texas,BarRestrict,1,NaN,"Deaf Smith, El Paso, Moore, Potter, and Randal...",https://lrl.texas.gov/scanned/govdocs/Greg%20A...,20210118,NaN,1.0,5,29,6.0,3.0
1736,TX,Texas,BarRestrict,1,NaN,Requires bars to close for on-premises consump...,https://gov.texas.gov/uploads/files/press/EO-G...,20210118,NaN,1.0,6,26,10.0,14.0


In [27]:
ons = np.zeros(12)
inds = []

for i in range(Tx_df.shape[0]):
    row = Tx_df.iloc[i]
    inds = inds + list(np.arange(row.EnactedMonth, row.EndedMonth + 1))
    print(np.arange(row.EnactedMonth - 1, row.EndedMonth))

inds = np.array(list(set(inds)))
inds = inds.astype('int')

ons[inds] = 1

ons

[2. 3. 4.]
[4. 5.]
[5. 6. 7. 8. 9.]


array([0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.])